# MLflow Model Registry — P2T2 모델 버전 관리

> **Purpose:** P2T2 파이프라인의 AI 모델을 Unity Catalog Model Registry에 등록합니다.
>
> **등록 모델:**
> - `P2T2.ai_results.soap_generator` — Azure OpenAI GPT-5.1 SOAP 노트 생성
> - `P2T2.ai_results.biomedclip_matcher` — BioMedCLIP 영상-텍스트 매칭
> - `P2T2.ai_results.judge_evaluator` — LLM-as-a-Judge 품질 평가

## 0. 설정

In [ ]:

import mlflow
import mlflow.pyfunc
from mlflow.tracking import MlflowClient
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec
from datetime import datetime
import json
import pandas as pd

spark.sql("USE CATALOG P2T2")
client = MlflowClient()

# Unity Catalog 3단계 이름: catalog.schema.model
REGISTRY = {
    "soap": "P2T2.ai_results.soap_generator",
    "clip": "P2T2.ai_results.biomedclip_matcher",
    "judge": "P2T2.ai_results.judge_evaluator",
}

# Unity Catalog 필수: 입출력 Signature
input_schema = Schema([ColSpec("string", "patient_id"), ColSpec("string", "input_text")])
output_schema = Schema([ColSpec("string", "result")])
SIGNATURE = ModelSignature(inputs=input_schema, outputs=output_schema)

# Unity Catalog Model Registry 사용 설정
mlflow.set_registry_uri("databricks-uc")

print("🔧 Model Registry 설정 완료")
print(f"   Tracking URI: {mlflow.get_tracking_uri()}")
print(f"   Registry URI: {mlflow.get_registry_uri()}")

## 1. PyFunc 래퍼 클래스 정의

In [ ]:

class SOAPGeneratorModel(mlflow.pyfunc.PythonModel):
    """Azure OpenAI 기반 SOAP 노트 생성 모델 래퍼"""
    def load_context(self, context):
        self.model_version = "gpt-51-deploy"
    def predict(self, context, model_input):
        results = []
        for _, row in model_input.iterrows():
            results.append({"result": f"SOAP note for {row.get('patient_id', 'unknown')}"})
        return pd.DataFrame(results)

class BioMedCLIPModel(mlflow.pyfunc.PythonModel):
    """BioMedCLIP 영상-텍스트 매칭 모델 래퍼"""
    def load_context(self, context):
        self.model_name = "BiomedCLIP-PubMedBERT_256"
    def predict(self, context, model_input):
        results = []
        for _, row in model_input.iterrows():
            results.append({"result": f"BioMedCLIP match for {row.get('patient_id', 'unknown')}"})
        return pd.DataFrame(results)

class JudgeEvaluatorModel(mlflow.pyfunc.PythonModel):
    """LLM-as-a-Judge 평가 모델 래퍼"""
    def load_context(self, context):
        self.judge_model = "gpt-51-deploy"
    def predict(self, context, model_input):
        results = []
        for _, row in model_input.iterrows():
            results.append({"result": f"Judge eval for {row.get('patient_id', 'unknown')}"})
        return pd.DataFrame(results)

print("✅ PyFunc 래퍼 클래스 3개 정의 완료")

## 2. 모델 로깅 및 Registry 등록

In [ ]:

mlflow.set_experiment("/Shared/P2T2_Model_Registry")

registered_models = {}

# ── 1. SOAP Generator ──
with mlflow.start_run(run_name="soap_generator_registration"):
    df_soap = spark.table("P2T2.ai_results.openai_soap_notes")
    mlflow.log_params({"model_name": "gpt-51-deploy", "api_version": "2025-04-01-preview", "temperature": "0.3"})
    mlflow.log_metrics({"total_notes_generated": df_soap.count()})
    mlflow.set_tags({"project": "P2T2", "model_category": "llm_soap"})
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=SOAPGeneratorModel(),
        signature=SIGNATURE,
        registered_model_name=REGISTRY["soap"],
    )
    registered_models["soap"] = mlflow.active_run().info.run_id
    print(f"✅ {REGISTRY['soap']} 등록 완료")

# ── 2. BioMedCLIP ──
with mlflow.start_run(run_name="biomedclip_registration"):
    df_clip = spark.table("P2T2.ai_results.biomedclip_results")
    mlflow.log_params({"model_name": "BiomedCLIP-PubMedBERT_256", "embedding_dim": "512", "image_size": "224x224"})
    mlflow.log_metrics({"total_matches": df_clip.count()})
    mlflow.set_tags({"project": "P2T2", "model_category": "multimodal_clip"})
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=BioMedCLIPModel(),
        signature=SIGNATURE,
        registered_model_name=REGISTRY["clip"],
    )
    registered_models["clip"] = mlflow.active_run().info.run_id
    print(f"✅ {REGISTRY['clip']} 등록 완료")

# ── 3. Judge Evaluator ──
with mlflow.start_run(run_name="judge_evaluator_registration"):
    df_judge = spark.table("P2T2.ai_results.judge_evaluation")
    mlflow.log_params({"judge_model": "gpt-51-deploy", "criteria": "accuracy,completeness,safety,actionability,relevance", "scoring_scale": "1-5"})
    mlflow.log_metrics({"total_evaluations": df_judge.count()})
    mlflow.set_tags({"project": "P2T2", "model_category": "llm_judge"})
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=JudgeEvaluatorModel(),
        signature=SIGNATURE,
        registered_model_name=REGISTRY["judge"],
    )
    registered_models["judge"] = mlflow.active_run().info.run_id
    print(f"✅ {REGISTRY['judge']} 등록 완료")

print(f"\n📦 총 {len(registered_models)}개 모델 등록 완료")

## 3. 모델 Alias 설정

In [ ]:

for key, name in REGISTRY.items():
    try:
        versions = client.search_model_versions(f"name='{name}'")
        if not versions:
            print(f"⚠️ {name}: 버전 없음")
            continue
        latest = max(versions, key=lambda v: int(v.version))
        client.set_registered_model_alias(name=name, alias="staging", version=latest.version)
        print(f"✅ {name} v{latest.version} → @staging")
    except Exception as e:
        print(f"ℹ️ {name}: alias 스킵 ({e})")

## 4. Registry 현황 리포트

In [ ]:

print("=" * 70)
print("📦 P2T2 MLflow Model Registry 현황")
print("=" * 70)

for key, name in REGISTRY.items():
    try:
        model = client.get_registered_model(name)
        versions = client.search_model_versions(f"name='{name}'")
        print(f"\n  🔹 {name}")
        print(f"     총 버전: {len(versions)}")
        for v in versions:
            aliases = v.aliases if hasattr(v, 'aliases') else []
            alias_str = f" (@{', @'.join(aliases)})" if aliases else ""
            print(f"     v{v.version}: {v.status}{alias_str}")
    except Exception:
        print(f"\n  ⚪ {name}: 미등록")

print(f"\n{'=' * 70}")
print("  ✅ 완료! Databricks UI → Models 탭에서 확인 가능")
print("=" * 70)